### Decorator Application : Single Dispatch Generic Functions

In [2]:
from html import escape

In [19]:
def html_escape(arg):
    return escape(str(arg))

def html_int(a):
    return '{0}(<i>{1}</i>)'.format(a,(hex(a)))

def html_real(a):
    return f'{round(a,2):.2f}'

def html_str(s):
    return html_escape(s).replace('\n','<br/>\n')

def html_list(l):
    items = ('<li>{0}'.format(htmlize(item))
            for item in l)
    return '<ul>\n'+'\n'.join(items)+'\n</ul>'

def html_dict(d):
    items = ('<li>{0}={1}</li>'.format(html_escape(k),htmlize(v))
            for k, v in d.items())
    return '<ul\n>'+'\n'.join(items)+'\n</ul>'

In [6]:
print(html_str("""this is 
a multiline string with 
special characters: 10 < 100"""))

this is <br/>
a multiline string with <br/>
special characters: 10 &lt; 100


In [7]:
print(html_int(255))

255(<i>0xff</i>)


In [9]:
print(html_escape(3+10j))

(3+10j)


In [10]:
from decimal import Decimal

In [20]:
def htmlize(arg):
    if isinstance(arg, int):
        return html_int(arg)
    elif isinstance(arg,float) or isinstance(arg,Decimal):
        return html_real(arg)
    elif isinstance(arg,str):
        return html_str(arg)
    elif isinstance(arg,list) or isinstance(arg,tuple):
        return html_list(arg)
    elif isinstance(arg,dict):
        return html_dict(arg)
    else:
        return html_escape(arg)

In [12]:
htmlize(100)

'100(<i>0x64</i>)'

In [13]:
htmlize("""Python
rocks""")

'Python<br/>\nrocks'

In [14]:
htmlize([1,2,3,4])

'<ul>\n<li>1\n<li>2\n<li>3\n<li>4\n</ul>'

In [23]:
print(htmlize(["""Python 
rocks! 0 < 1""",(10,20,30),100]))

<ul>
<li>Python <br/>
rocks! 0 &lt; 1
<li><ul>
<li>10(<i>0xa</i>)
<li>20(<i>0x14</i>)
<li>30(<i>0x1e</i>)
</ul>
<li>100(<i>0x64</i>)
</ul>


In [32]:
def htmlize(arg):
    registry = {
        object: html_escape,
        int: html_int,
        float: html_real,
        Decimal: html_real,
        str: html_str,
        list: html_list,
        tuple: html_list,
        dict: html_dict
    }
    
    fn = registry.get(type(arg),registry[object])
    return fn(arg)

In [70]:
def single_dispatch(fn):
    registry = {}
    registry[object] = fn
    def decorated(arg):
        return registry.get(type(arg),registry[object])(arg)
    
    def register(type_):
        def inner(fn):
            registry[type_] = fn
            return fn
        return inner
    
    def dispatch(type_):
        return registry.get(type_,registry[object])
    decorated.register = register   
   # decorated.registry = registry # useful to see whats in your dictionary but not very safe
    decorated.dispatch = dispatch
    return decorated

In [71]:
@single_dispatch
def htmlize(a):
    return escape(str(a))

In [72]:
htmlize('1 < 100')

'1 &lt; 100'

In [73]:
@htmlize.register(int)
def html_int(a):
    return '{0}(<i>{1}</i>)'.format(a,str((hex(a))))

In [74]:
htmlize(100)

'100(<i>0x64</i>)'

In [75]:
@htmlize.register(list)
def html_list(l):
    items = ('<li>{0}'.format(htmlize(item))
            for item in l)
    return '<ul>\n'+'\n'.join(items)+'\n</ul>'

In [68]:
htmlize([1,2,3,4])

'<ul>\n<li>1(<i>0x1</i>)\n<li>2(<i>0x2</i>)\n<li>3(<i>0x3</i>)\n<li>4(<i>0x4</i>)\n</ul>'

In [76]:
htmlize.dispatch(int)

<function __main__.html_int(a)>

In [77]:
from numbers import Integral

# isinstance() vs type()

In [78]:
class Person:
    pass

In [79]:
class Student(Person):
    pass

In [80]:
p = Student()

In [82]:
type(p)

__main__.Student

In [83]:
isinstance(p,Student)

True

In [84]:
isinstance(p,Person)

True

In [85]:
@single_dispatch
def htmlize(a):
    return escape(str(a))

In [86]:
@htmlize.register(Integral)
def html_integral_number(a):
    return '{0}(<i>{1}</i>)'.format(a,str(hex(a)))

In [87]:
isinstance(10,Integral)

True

In [88]:
isinstance(True,Integral)

True

In [89]:
@htmlize.register(int)
@htmlize.register(bool)
def html_integral_number(a):
    return '{0}(<i>{1}</i>)'.format(a,str(hex(a)))

In [90]:
htmlize(10)

'10(<i>0xa</i>)'

In [91]:
htmlize(True)

'True(<i>0x1</i>)'

In [92]:
from collections.abc import Sequence

In [93]:
isinstance([1,2,3],Sequence)

True

SyntaxError: unexpected EOF while parsing (<ipython-input-95-068eb44a1ae0>, line 1)

In [99]:
from functools import singledispatch

In [100]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [101]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>})

In [102]:
htmlize.dispatch(str)

<function __main__.htmlize(a)>

In [103]:
@htmlize.register(Integral)
def htmlize_integral_number(a):
    return '{0}(<i>{1}</i>)'.format(a,str(hex(a)))

In [104]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>,
              numbers.Integral: <function __main__.htmlize_integral_number(a)>})

In [105]:
htmlize.dispatch(int)

<function __main__.htmlize_integral_number(a)>

In [106]:
isinstance(10,Integral)

True

In [107]:
isinstance(True,Integral)

True

In [108]:
htmlize(10)

'10(<i>0xa</i>)'

In [109]:
htmlize(True)

'True(<i>0x1</i>)'

In [112]:
@htmlize.register(Sequence)
def html_list(l):
    items = ('<li>{0}'.format(htmlize(item))
            for item in l)
    return '<ul>\n'+'\n'.join(items)+'\n</ul>'

In [113]:
htmlize([1,2,3,4])

'<ul>\n<li>1(<i>0x1</i>)\n<li>2(<i>0x2</i>)\n<li>3(<i>0x3</i>)\n<li>4(<i>0x4</i>)\n</ul>'

In [114]:
htmlize((1,2,3))

'<ul>\n<li>1(<i>0x1</i>)\n<li>2(<i>0x2</i>)\n<li>3(<i>0x3</i>)\n</ul>'